In [44]:
import numpy as np
import sklearn
import pandas

In [70]:
tracking_data = np.load("compressed_final_data/copulation_raw_tracking_data.npz")["data"]
label_data = np.load("compressed_final_data/copulation_raw_label_data.npz")["data"]

In [71]:
label_data

array([[False, False],
       [False, False],
       [False, False],
       ...,
       [False, False],
       [False, False],
       [False, False]])

In [72]:
np.sum(label_data)

9619

In [73]:
len(label_data)

16740

# testing

In [52]:
import scipy.io as sio
DIR = "fruitflydata/Courtship/"

raw_tracking_data = []
raw_label_data = []

SEQUENCE_LENGTH = 100
action = "copulation"


In [53]:
interval_file = "fruitflydata/Courtship/wild/movie1/movie1_actions.mat"
tracking_file = "fruitflydata/Courtship/wild/movie1/movie1_track.mat"

In [54]:
			temp = sio.loadmat(tracking_file)
			interval_file_data = sio.loadmat(interval_file)
			behs = interval_file_data["behs"][0]
			bouts = interval_file_data["bouts"]
			for index in range(len(behs)):
				if behs[index][0] == action:
					index_of_action = index

			fly_1_bouts = bouts[0][index_of_action]
			fly_2_bouts = bouts[1][index_of_action]

			# fly_1_bouts = np.sort(fly_1_bouts, axis=0)
			# fly_2_bouts = np.sort(fly_2_bouts, axis=0)
			fly_1_bouts_index = 0
			fly_2_bouts_index = 0

			index_of_action = None

			number_of_true_actions_fly_1 = 0
			number_of_true_actions_fly_2 = 0

			for k in range(len(fly_1_bouts)):
				number_of_true_actions_fly_1 += (fly_1_bouts[k][1] - fly_1_bouts[k][0])/SEQUENCE_LENGTH

			for k in range(len(fly_2_bouts)):
				number_of_true_actions_fly_2 += (fly_2_bouts[k][1] - fly_2_bouts[k][0])/SEQUENCE_LENGTH


			print("NUMBER OF FLY 1 ACTIONS : " + str(number_of_true_actions_fly_1))
			print("NUMBER OF FLY 2 ACTIONS : " + str(number_of_true_actions_fly_2))

NUMBER OF FLY 1 ACTIONS : 343.04
NUMBER OF FLY 2 ACTIONS : 0


In [56]:


			data = temp["trk"][0]

			flag_frames = data[0][0]
			names = data[0][1]
			tracking_data = data[0][2]

			positions_only = tracking_data[:, :, :2]

			final = []
			final_action_labels = []

			flies_switched = flag_frames[0]
			switched = False

In [65]:
def overlap_interval(a1, a2, b1, b2):
	return (b2 >= a1 and b2 <= a2) or (b1 >= a1 and b1 <= a2) or (b1 >= a1 and b2 <= a2) or (b1 <= a1 and b2 >= a2)

In [67]:
for j in range(0, positions_only.shape[1], SEQUENCE_LENGTH): 
    # assume flags_switched[j] == 1 means that from this frame on (inclusive) flies have switched
    print("going through indices " + str(j) + " to " + str(j + SEQUENCE_LENGTH))
    label_fly_1 = False
    label_fly_2 = False

    # check if fly_1 should be labeled as copulation
    
    for fly_1_t in range(len(fly_1_bouts)):
        print("checking overlap for fly sequence : " + str(j) + " to " + str(j + SEQUENCE_LENGTH) + " with fly bout as "+ str(fly_1_bouts[fly_1_t][0]) + " to " + str(fly_1_bouts[fly_1_t][1]))
        if overlap_interval(j, j + SEQUENCE_LENGTH, fly_1_bouts[fly_1_t][0], fly_1_bouts[fly_1_t][1]):
#             print("found overlap fly 1")
            label_fly_1 = True
            break

    for fly_2_t in range(len(fly_2_bouts)):
        if overlap_interval(j, j + SEQUENCE_LENGTH, fly_2_bouts[fly_2_t][0], fly_2_bouts[fly_2_t][1]):
#             print("found overlap fly 2")

            label_fly_2 = True
            break

    x_1 = positions_only[0, j: j + SEQUENCE_LENGTH, 0] 
    y_1 = positions_only[0, j: j + SEQUENCE_LENGTH, 1] 
    x_2 = positions_only[1, j: j + SEQUENCE_LENGTH, 0] 
    y_2 = positions_only[1, j: j + SEQUENCE_LENGTH, 1] 

    add_arr = np.array([x_1, y_1, x_2, y_2]) 
    add_arr = np.transpose(add_arr)
    add_arr_length = add_arr.shape[0]
    if (add_arr_length < SEQUENCE_LENGTH):
        last_row = add_arr[add_arr_length - 1]
        add_arr = np.vstack((add_arr, last_row))

    # print(add_arr.shape)
    final.append(add_arr) 
    final_action_labels.append([label_fly_1, label_fly_2])


if (len(raw_tracking_data) == 0):
    raw_tracking_data = np.array(final)
    raw_label_data = np.array(final_action_labels)
else:
    raw_tracking_data = np.vstack((raw_tracking_data, final))
    raw_label_data = np.vstack((raw_label_data, final_action_labels))

going through indices 0 to 100
checking overlap for fly sequence : 0 to 100 with fly bout as 13417 to 47721
going through indices 100 to 200
checking overlap for fly sequence : 100 to 200 with fly bout as 13417 to 47721
going through indices 200 to 300
checking overlap for fly sequence : 200 to 300 with fly bout as 13417 to 47721
going through indices 300 to 400
checking overlap for fly sequence : 300 to 400 with fly bout as 13417 to 47721
going through indices 400 to 500
checking overlap for fly sequence : 400 to 500 with fly bout as 13417 to 47721
going through indices 500 to 600
checking overlap for fly sequence : 500 to 600 with fly bout as 13417 to 47721
going through indices 600 to 700
checking overlap for fly sequence : 600 to 700 with fly bout as 13417 to 47721
going through indices 700 to 800
checking overlap for fly sequence : 700 to 800 with fly bout as 13417 to 47721
going through indices 800 to 900
checking overlap for fly sequence : 800 to 900 with fly bout as 13417 to 47

In [66]:
overlap_interval(14900, 15000, 13417, 47721)

True

In [69]:
np.sum(raw_label_data)

362

array([], shape=(0, 3), dtype=uint8)